# Reconstruct the SBML from a set of reactions

This is to test the FBA toolchain

In [1]:
import sys
import os
import copy
import PyFBA
import re
import pickle

import inspect
inspect.getfile(PyFBA)

'/home/redwards/GitHubsLinux/PyFBA/venv/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/__init__.py'

# Read the Reactions, Enzymes, and Compounds Sets

This parses the ModelData data

In [2]:
modelseed = PyFBA.parse.model_seed.parse_model_seed_data('gramnegative', verbose=True)
print(f"There are {len(modelseed.compounds):,} compounds, {len(modelseed.reactions):,} reactions, and {len(modelseed.enzymes):,} enzymes in total")

We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/PyFBA.2021-06-10T21:50:17.900873.log
Reading compounds from PyFBA.Biochemistry.ModelSEEDDatabase.Biochemistry.compounds.json


There are 33,992 compounds, 43,774 reactions, and 9,423 enzymes in total


# Create a biomass equation

In [3]:
biomass_equation = PyFBA.metabolism.biomass_equation('gramnegative', modelseed.compounds)

# Read the media and correct the names

In [5]:
# Read the media file
media = PyFBA.parse.pyfba_media("ArgonneLB")
# Correct the names
media = PyFBA.parse.correct_media_names(media, modelseed.compounds)
print(f"The media has {len(media)} compounds")

The media has 65 compounds


#TODO:

1. Are the compounds in the media the same as the compounds in the reactions?
2. Are the compounds in the reactions the same as the compounds in modelseed.compounds?
    a. What does the SM use as compounds in the columns?
    b. What does the SM use as reactions in the rows?
3. Are the reactions_to_run the same as the reactions?
4. Are the compounds in the biomass_equation the same as the compounds in the media/reactions/modelseed.compounds?

# Read the reactions

In [6]:
reactions_to_run = set()
with open('rgood.txt', 'r') as f:
    for l in f:
        if l.startswith('rxn'):
            reactions_to_run.add(l.strip())
print(f"There are {len(reactions_to_run)} reactions to run")

There are 1399 reactions to run


# Add the biomass equation to the reactions to run

In [7]:
#reactions_to_run.add(sbml_biomass)

In [8]:
# for r in reactions_to_run:
#     if r not in modelseed.reactions:
#         sys.stderr.write(f"Error: reaction {r} not found\n")

# Run the FBA

In [9]:
uptake_secretion_reactions = set()

In [10]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


create_stoichiometric_matrix is adding compound cpd00644: PAN (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00971: Na+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00046: CMP (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00161: L-Threonine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00028: Heme (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00092: Uracil (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00246: Inosine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00129: L-Proline (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd01048: Arsenate (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00184: Thymidine (loca

We are loading 1324 rows and 1592 columns


The FBA completed with a flux value of 0.0 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1324
Number of reactions in SM: 1592
Revised number of total reactions: 43966
Number of total compounds: 34117
SMat dimensions: 1324 x 1592


## Gapfilling the reactions

In [11]:
added_reactions = []
reactionsource = {r: "original" for r in reactions_to_run}

In [12]:
def gap_fill_media():
    sys.stderr.write("Gap filling from MEDIA\n")
    media_reactions = PyFBA.gapfill.suggest_from_media(modelseed.compounds, modelseed.reactions, reactions_to_run, media, verbose=True)
    added_reactions.append(("media", media_reactions))
    reactions_to_run.update(media_reactions)
    status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
    sys.stderr.write(f"After adding {len(media_reactions)} MEDIA reactions we get {value} (growth is {growth})\n")
    for r in media_reactions:
        if r not in reactionsource:
            reactionsource[r] = 'media_reactions'

    if growth:
        additions = resolve_additional_reactions(original_reactions, added_reactions, compounds, reactions,
                                                 media, biomass_eqtn)
        # print('reactions' + " : " + str(original_reactions.union(additions)))
        for r in original_reactions.union(additions):
            if r not in reactionsource:
                reactionsource[r] = "UNKNOWN??"
            print("{}\t{}".format(r, reactionsource[r]))

In [13]:
def essential_reactions():
    sys.stderr.write("Gap filling from ESSENTIAL PROTEINS\n")
    essential_reactions = PyFBA.gapfill.suggest_essential_reactions()
    # find only the new reactions
    essential_reactions.difference_update(reactions_to_run)
    added_reactions.append(("essential", essential_reactions))
    reactions_to_run.update(essential_reactions)
    status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                      reactions_to_run, media, biomass_equation,
                                      uptake_secretion_reactions, verbose=True)
    sys.stderr.write("After adding {} ESSENTIAL reactions we get {} (growth is {})\n\n".format(len(essential_reactions),
                                                                                               value, growth))

    for r in essential_reactions:
        if r not in reactionsource:
            reactionsource[r] = 'essential_reactions'

    # if this grows then we want to find the minimal set of reactions
    # that we need to add for growth and call it good.
    if growth:
        additions = resolve_additional_reactions(original_reactions, added_reactions, modelseed.compounds, modelseed.reactions,
                                                 media, biomass_equation)
        for r in original_reactions.union(additions):
            if r not in reactionsource:
                reactionsource[r] = "UNKNOWN??"
            print("{}\t{}".format(r, reactionsource[r]))

In [14]:
def close_genomes(close_genomes=None, similar_genera=None):
    sys.stderr.write("Gap filling from CLOSE GENOMES\n")
    close_reactions = set()
    if close_genomes:
        # add reactions from roles in close genomes
        close_reactions = PyFBA.gapfill.suggest_from_roles(close_genomes, modelseed.reactions, threshold=0, verbose=True)
        # find the new reactions
        close_reactions.difference_update(reactions_to_run)
        added_reactions.append(("close genomes ", close_reactions))
        reactions_to_run.update(close_reactions)
        status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                  reactions_to_run, media, biomass_equation,
                                  uptake_secretion_reactions, verbose=True)
        
        sys.stderr.write(f"After adding {len(close_reactions)} reactions in {close_genomes} we get {value} (growth is {growth})\n")
        
        for r in close_reactions:
            if r not in reactionsource:
                reactionsource[r] = 'close_reactions'

        # if this grows then we want to find the minimal set of reactions
        # that we need to add for growth and call it good.
        if growth:
            additions = resolve_additional_reactions(original_reactions, added_reactions, modelseed.compounds, modelseed.reactions,
                                                     media, biomass_equation)
            # print("Additional reactions required: " + str(additions) + "\n")
            # print("'reactions': {}".format(original_reactions.union(additions)))
            for r in original_reactions.union(additions):
                if r not in reactionsource:
                    reactionsource[r] = "UNKNOWN??"
                print("{}\t{}".format(r, reactionsource[r]))

    genus_reactions = set()
    if similar_genera:
        # add reactions from roles in similar genera
        genus_reactions = PyFBA.gapfill.suggest_from_roles(similar_genera, reactions, threshold=0, verbose=True)
        # find the new reactions
        genus_reactions.difference_update(reactions_to_run)
        added_reactions.append(("other genera", genus_reactions))
        reactions_to_run.update(genus_reactions)
        status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                          reactions_to_run, media, biomass_equation,
                          uptake_secretion_reactions, verbose=True)        
        sys.stderr.write(f"After adding {len(genus_reactions)} reactions in {similar_genera} we get {value} (growth is {growth})\n")

        for r in genus_reactions:
            if r not in reactionsource:
                reactionsource[r] = 'genus_reactions'

        # if this grows then we want to find the minimal set of reactions
        # that we need to add for growth and call it good.
        if growth:
            additions = resolve_additional_reactions(original_reactions, added_reactions, modelseed.compounds, modelseed.reactions,
                                                     media, biomass_equation)
            for r in original_reactions.union(additions):
                if r not in reactionsource:
                    reactionsource[r] = "UNKNOWN??"
                print("{}\t{}".format(r, reactionsource[r]))


In [15]:
def gapfill_subsystems():
    sys.stderr.write("Gap filling from SUBSYSTEMS\n")
    subsystem_reactions = PyFBA.gapfill.suggest_reactions_from_subsystems(
        modelseed.reactions, reactions_to_run, threshold=0.5, verbose=True)
    added_reactions.append(("subsystems", subsystem_reactions))
    reactions_to_run.update(subsystem_reactions)
    status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions, reactions_to_run, media, biomass_equation)
    sys.stderr.write(f"After adding {len(subsystem_reactions)} SUBSYSTEM reactions we get {value} (growth is {growth})\n")
    for r in subsystem_reactions:
        if r not in reactionsource:
            reactionsource[r] = 'subsystem_reactions'

    if growth:
        additions = resolve_additional_reactions(original_reactions, added_reactions, modelseed.compounds, modelseed.reactions,
                                                 media, biomass_equation)
        # print('reactions' + " : " + str(original_reactions.union(additions)))
        for r in original_reactions.union(additions):
            if r not in reactionsource:
                reactionsource[r] = "UNKNOWN??"
            print("{}\t{}".format(r, reactionsource[r]))

In [16]:
essential_reactions()

Gap filling from ESSENTIAL PROTEINS
create_stoichiometric_matrix found 192 uptake and secretion reactions
In the model there are : 1325 compounds and 1595 reactions
We are loading 1325 rows and 1595 columns
Length of the media: 65
Number of reactions to run: 1402
Number of compounds in SM: 1325
Number of reactions in SM: 1595
Revised number of total reactions: 43966
Number of total compounds: 34117
SMat dimensions: 1325 x 1595
After adding 3 ESSENTIAL reactions we get 0.0 (growth is False)



In [17]:
gapfill_subsystems()

Gap filling from SUBSYSTEMS
Too few columns in subsystem file at line: ARGININE-SYN
Too few columns in subsystem file at line: NAD_and_NADP_-_test_subsystem
Too few columns in subsystem file at line:


Maltose_and_Maltodextrin_Utilization	0.3333333333333333	{'rxn01132', 'rxn08128', 'rxn08941', 'rxn05607', 'rxn08127', 'rxn05147', 'rxn05608', 'rxn08126', 'rxn08129', 'rxn05170', 'rxn08942', 'rxn00022', 'rxn05226', 'rxn10174', 'rxn05740', 'rxn01171', 'rxn01967', 'rxn08943', 'rxn01966', 'rxn01133', 'rxn05173'}
Trehalose_Uptake_and_Utilization	0.3333333333333333	{'rxn05608', 'rxn01132', 'rxn05226', 'rxn05170', 'rxn10174', 'rxn01966', 'rxn02005', 'rxn00606', 'rxn05607', 'rxn00007', 'rxn01967'}
Trehalose_Biosynthesis	0.36363636363636365	{'rxn01132', 'rxn01966', 'rxn02004', 'rxn00605', 'rxn01134'}
Sugar_catabolome_in_Shewanella_species	0.3409090909090909	{'rxn01132', 'rxn05501', 'rxn04082', 'rxn00704', 'rxn01102', 'rxn02005', 'rxn00785', 'rxn01200', 'rxn00295', 'rxn00558', 'rxn05560', 'rxn01967', 'rxn01951', 'rxn00552', 'rxn01966', 'rxn01859', 'rxn01146', 'rxn02380', 'rxn01106', 'rxn01799', 'rxn05607', 'rxn05655', 'rxn05647', 'rxn01484', 'rxn01477', 'rxn00022', 'rxn01986', 'rxn01331', 'rxn011

Lipid_A-Ara4N_pathway_(_Polymyxin_resistance_)	0.18181818181818182	{'rxn00692', 'rxn00211'}
Streptococcal_Hyaluronic_Acid_Capsule	0.6666666666666666	{'rxn00213', 'rxn00211'}
Streptococcal_group_antigen_operons	0.16666666666666666	{'rxn00214', 'rxn03907', 'rxn02003', 'rxn01997', 'rxn00295'}
D-Sorbitol(D-Glucitol)_and_L-Sorbose_Utilization	0.13333333333333333	{'rxn03886', 'rxn10184'}
WhiB_and_WhiB-type_regulatory_proteins	0.125	{'rxn10199', 'rxn00834'}
WhiB_and_WhiB-type_regulatory_proteins_	0.125	{'rxn10199', 'rxn00834'}
Peptidoglycan_biosynthesis--gjo	0.3076923076923077	{'rxn02011', 'rxn02286', 'rxn02008', 'rxn03164'}
Coenzyme_PQQ_synthesis	0.15	{'rxn10770', 'rxn10116'}
PTPS_paralogs	0.2	{'rxn00686', 'rxn02504', 'rxn02201', 'rxn02503', 'rxn01602', 'rxn02200'}
COG4319	0.7142857142857143	{'rxn00686', 'rxn02504', 'rxn00691', 'rxn02201', 'rxn02503', 'rxn01602', 'rxn02200'}
Folate_test_GB	0.6666666666666666	{'rxn00689', 'rxn01603', 'rxn00686', 'rxn02504', 'rxn02201', 'rxn02503', 'rxn01602',

Subsystems suggesting 469 subsystems
Subsystems suggesting 2655 roles
Subsystems suggesting 1477 reactions
Subsystems suggested 1608 reactions
create_stoichiometric_matrix found 211 uptake and secretion reactions
After adding 431 SUBSYSTEM reactions we get 0.0 (growth is False)


In [18]:
# gap_fill_media()

## Comparing to the SBML file that does grow!

We pickled that data, and can load it here. 

In [19]:
sbml_filtered_compounds = pickle.load(open('compounds.pickle', 'rb'))
sbml_reactions = pickle.load(open('reactions.pickle', 'rb'))
sbml_reactions_to_run = pickle.load(open('reactions_to_run.pickle', 'rb'))
sbml_media = pickle.load(open('media.pickle', 'rb'))
sbml_biomass_equation = pickle.load(open('sbml_biomass.pickle', 'rb'))
sbml_uptake_secretion_reactions = pickle.load(open('uptake_secretion_reactions.pickle', 'rb'))

In [20]:
for r in sbml_reactions:
    for c in sbml_reactions[r].all_compounds():
        if '_' in c.name:
            print(c.name)

L_Alanine
L_Methionine
met_L_ala_L
L_Alanine
ala_L_Thr_L
L_Threonine
L_Aspartate
gly_asp_L
L_Cysteine
Gly_Cys
L_Alanine
L_Glutamate
ala_L_glu_L
gly_asn_L
L_Asparagine
L_Leucine
Gly_Leu
L_Alanine
L_alanylglycine
L_Alanine
L_Aspartate
ala_L_asp_L
L_Cysteine
Cys_Gly
L_Glutamine
Gly_Gln
L_Phenylalanine
Gly_Phe
Ala_His
L_Histidine
L_Alanine
gly_glu_L
L_Glutamate
L_Alanine
L_Glutamine
Ala_Gln
Gly_Tyr
L_Tyrosine
L_Proline
gly_pro_L
L_Leucine
Ala_Leu
L_Alanine
Gly_Met
L_Methionine
Glucose_1_phosphate
Crotonyl_CoA
Butyryl_CoA
Methacrylyl_CoA
Isobutyryl_CoA
Tiglyl_CoA
2_Methylbutyryl_CoA
N_Acetyl_D_glucosamine1_phosphate
D_Glucosamine1_phosphate
Acetyl_CoA
N_Acetyl_D_glucosamine1_phosphate
UDP_N_acetylglucosamine
L_Arginine
L_Argininosuccinate
3_oxoadipate_enol_lactone
4_Carboxymuconolactone
2_Demethylmenaquinol_8
2_Demethylmenaquinone_8
Ubiquinol_8
Ubiquinone_8
Menaquinone_8
Menaquinol_8
1_2_dianteisoheptadecanoyl_sn_glycerol_3_phosphate
1_anteisoheptadecanoyl_sn_glycerol_3_phosphate
1_isohexad

Hexadecenoyl_CoA
alpha_Ribazole_5_phosphate
Nicotinate_ribonucleotide
5_Deoxy_glucuronic_acid
L_Malate
Acetyl_CoA
ADP_D_glycero_D_manno_heptose
D_Glycero_D_manno_heptose1_phosphate
D_Tagatose_6_phosphate
Galactitol_1_phosphate
Vitamin_K1
S_Adenosyl_L_methionine
S_Adenosyl_homocysteine
2_Octaprenyl_3_methyl_6_methoxy_1_4_benzoquinone
2_Octaprenyl_6_methoxy_1_4_benzoquinone
S_Adenosyl_L_methionine
S_Adenosyl_homocysteine
Menaquinone_8
2_Demethylmenaquinone_8
S_Adenosyl_L_methionine
S_Adenosyl_homocysteine
Protein_biosynthesis
5_Methyltetrahydrofolate
5_10_Methylenetetrahydrofolate
5_Methyltetrahydrofolate
5_10_Methylenetetrahydrofolate
L_Alanine
UDP_N_acetylmuramoyl_L_alanine
UDP_MurNAc
UDP_2_3_bis3_hydroxytetradecanoylglucosamine
Lipid_A_disaccharide
Lipid_X
Glyceraldehyde3_phosphate
Indoleglycerol_phosphate
L_Serine
L_Tryptophan
Glyceraldehyde3_phosphate
Indoleglycerol_phosphate
L_Serine
L_Tryptophan
5_Amino_6__5_phosphoribosylaminouracil
5_Amino_6__5_phosphoribitylaminouracil
L_Ribulo

Gly_Tyr
Gly_Tyr
Ala_Leu
Ala_Leu
D_Fructose
D_Fructose
met_L_ala_L
met_L_ala_L
L_Asparagine
L_Asparagine
Ala_His
Ala_His
L_Lysine
L_Lysine
N_Acetyl_D_glucosamine
N_Acetyl_D_glucosamine
D_Mucic_acid
D_Mucic_acid
L_Histidine
L_Histidine
D_Glucose
D_Glucose
L_Serine
L_Serine
L_Phenylalanine
L_Phenylalanine
L_Aspartate
L_Aspartate
gly_glu_L
gly_glu_L
L_Leucine
L_Leucine
L_Proline
L_Proline
D_Methionine
D_Methionine
D_Arabinose
D_Arabinose
Vitamin_B12
Vitamin_B12
Myristic_acid
Myristic_acid
gly_pro_L
gly_pro_L
L_Valine
L_Valine
D_Galactonate
D_Galactonate
D_Alanine
D_Alanine
L_Glutamate
L_Glutamate
Gly_Met
Gly_Met
D_Mannose
D_Mannose
D_Glucarate
D_Glucarate
Gly_Phe
Gly_Phe
L_Arginine
L_Arginine
Glycerol_3_phosphate
Glycerol_3_phosphate
2_Oxoglutarate
2_Oxoglutarate
L_Rhamnose
L_Rhamnose
Gly_Leu
Gly_Leu
Gly_Cys
Gly_Cys
L_Inositol
L_Inositol
D_mannose_6_phosphate
D_mannose_6_phosphate
5_Oxoproline
5_Oxoproline
D_glucose_6_phosphate
D_glucose_6_phosphate
L_Alanine
L_Alanine


In [21]:
sbml_by_equation = set()
for r in sbml_reactions:
    sbml_by_equation.add(sbml_reactions[r].equation)
ms_by_equation = set()
for r in modelseed.reactions:
    ms_by_equation.add(modelseed.reactions[r].equation)


In [22]:
print(f"There are {len(sbml_by_equation)} sbml reactions and {len(ms_by_equation)} model seed reactions")
print(f"{len(sbml_by_equation.intersection(ms_by_equation))} reactions are in both")

There are 1764 sbml reactions and 36920 model seed reactions
98 reactions are in both


In [23]:
print(f"There are {len(sbml_reactions_to_run)} sbml reactions to run and {len(reactions_to_run)} model seed reactions to run")
print(f"{len(sbml_reactions_to_run.intersection(reactions_to_run))} are the same")


There are 1573 sbml reactions to run and 1833 model seed reactions to run
1399 are the same


In [24]:
# choose an sbml reaction to run at random
from random import sample

In [25]:
rr = sample(list(sbml_reactions_to_run), 1)[0]
if rr in reactions_to_run:
    print(rr)
    print(sbml_reactions[rr].equation)
    print(modelseed.reactions[rr].equation)

rxn02596
 (1) cpd00001: H2O (location: c) +  (1) cpd01133: Stachyose (location: c) >  (1) cpd00382: Melitose (location: c) +  (1) cpd00108: Galactose (location: c)
(1) H2O[c] + (1) Stachyose[c] <=> (1) Galactose[c] + (1) Melitose[c]


In [39]:
def fixname(x):
    x = x.replace('-', '_')
    x = x.replace(',', '_')
    x = x.replace('sn_glycerol 3_phosphate', 'sn_glycerol_3_phosphate')
    return x

sbml_rewritten = {}
for rx in sbml_reactions:
    l = ""
    for c in sorted(sbml_reactions[rx].left_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        l += f" + ({sbml_reactions[rx].get_left_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    l = l.replace(' + ', '', 1)
    r = ""
    for c in sorted(sbml_reactions[rx].right_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        r += f" + ({sbml_reactions[rx].get_right_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    r = r.replace(' + ', '', 1)
    e = f"{l} {sbml_reactions[rx].direction} {r}"
    sbml_rewritten[rx] = e

ms_rewritten = {}
for rx in modelseed.reactions:
    l = ""
    for c in sorted(modelseed.reactions[rx].left_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        l += f" + ({modelseed.reactions[rx].get_left_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    l = l.replace(' + ', '', 1)
    r = ""
    for c in sorted(modelseed.reactions[rx].right_compounds, key=lambda c:c.name.lower()):
        if c.name == 'H' or c.name == 'H+':
            continue
        r += f" + ({modelseed.reactions[rx].get_right_compound_abundance(c):.0f}) {fixname(c.name)}[{c.location}]"
    r = r.replace(' + ', '', 1)
    e = f"{l} {modelseed.reactions[rx].direction} {r}"
    ms_rewritten[rx] = e

In [40]:
for r in sbml_rewritten:
    if r in ms_rewritten:
        if sbml_rewritten[r] != ms_rewritten[r]:
            print(f"\n{r}\ns: {sbml_rewritten[r]}\nm: {ms_rewritten[r]}")


rxn00868
s: (1) Crotonyl_CoA[c] + (1) NADH[c] > (1) Butyryl_CoA[c] + (1) NAD[c]
m: (1) Butyryl_CoA[c] + (1) NAD[c] < (1) Crotonyl_CoA[c] + (1) NADH[c]

rxn08976
s: (1) 2_Demethylmenaquinone_8[c] + (1) NADH[c] > (1) 2_Demethylmenaquinol_8[c] + (1) NAD[c]
m: (1) 2_Demethylmenaquinone 8[c] + (1) NADH[c] > (1) 2_Demethylmenaquinol 8[c] + (1) NAD[c]

rxn08971
s: (1) Menaquinone_8[c] + (1) NADH[c] > (1) Menaquinol_8[c] + (1) NAD[c]
m: (1) Menaquinone 8[c] + (1) NADH[c] > (1) Menaquinol 8[c] + (1) NAD[c]

rxn05565
s: (1) D_Mucic_acid[e] = (1) D_Mucic_acid[c]
m: (1) D_Mucic acid[e] = (1) D_Mucic acid[c]

rxn00225
s: (1) Acetate[c] + (1) ATP[c] = (1) Acetylphosphate[c] + (1) ADP[c]
m: (1) Acetate[c] + (1) ATP[c] < (1) Acetylphosphate[c] + (1) ADP[c]

rxn00985
s: (1) ATP[c] + (1) Propionate[c] = (1) ADP[c] + (1) Propionyl_phosphate[c]
m: (1) ATP[c] + (1) Propionate[c] = (1) ADP[c] + (1) Propionyl phosphate[c]

rxn10770
s: (1) Choline[c] + (1) NAD[c] = (1) Betaine_aldehyde[c] + (1) NADH[c]
m: (1

So if we use the `sbml_reactions` and the `sbml_biomass_equation`, we get growth. Without those reactions we get an error. With the normal biomass equation, we don't get any growth

In [42]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          sbml_reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

KeyError: 'EX_cpd01012'

In [ ]:
for c in modelseed.compounds:
    if c.id == 'cpd00060':
        print(c)

### Note the normal biomass equation does not work!

We need to figure out what is missing!

In [ ]:
biomass_equation = PyFBA.metabolism.biomass_equation('gram_negative')
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_reactions,
                                          reactions_to_run, media, biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

### And the modelseed.reactions gives us an error (or no growth

In [ ]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r not in modelseed.reactions:
        print(r)

In [ ]:
tmpr = copy.deepcopy(modelseed.reactions)
tmpr['biomass_equation'] = sbml_reactions['biomass_equation']

In [ ]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, tmpr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
sbml_reactions['EX_cpd01022'].__dict__

In [ ]:
external = set()
for r in reactions_to_run:
    for c in modelseed.reactions[r].all_compounds():
        if c.location == 'e':
            external.add(c)
print(f"There are {len(external)} external compounds")

In [ ]:
newr = {}
for e in external:
    n = PyFBA.metabolism.Reaction(f"EX_{e.id}", readable_name='external rctn {e.id}',
                                 equation = f"(1) {e} = (1) {e}", direction='=')
    n.add_left_compounds(c)
    n.add_right_compounds(c)
    n.set_left_compound_abundance(c, 1)
    n.set_right_compound_abundance(c, 1)
    n.lower_bound = -1000
    n.upper_bound = 1000
    newr[f"EX_{e.id}"] = n


In [ ]:
mr = copy.deepcopy(modelseed.reactions)
mr.update(newr)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, mr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
mr['biomass_equation'] = sbml_reactions['biomass_equation']
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, mr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r not in mr:
        print(r)

In [ ]:
sbml_reactions['EX_cpd08636'].__dict__

In [ ]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_partial,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
del sbml_partial['EX_cpd11416']

In [ ]:
for f in ['EX_cpd15302', 'EX_cpd08636', 'EX_cpd02701']:
    del sbml_partial[f]

In [ ]:
for r in sbml_partial:
    if r not in mr:
        print(r)

In [ ]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_partial,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r in modelseed.reactions:
        left_diff = 0
        right_diff = 0
        for l in modelseed.reactions[r].left_compounds:
            try:
                a = sbml_reactions[r].get_left_compound_abundance(l)
            except KeyError:
                left_diff += 1
        for l in modelseed.reactions[r].right_compounds:
            try:
                a = sbml_reactions[r].get_right_compound_abundance(l)
            except KeyError:
                right_diff += 1
        if left_diff > 0 or right_diff > 0:
            print(f"{r}\t{left_diff}\t{right_diff}")

In [ ]:
for r in sbml_reactions:
    print(r)

In [ ]:
sbml_reactions['rxn01516'].__dict__

In [ ]:
modelseed.reactions['rxn01516'].__dict__

In [ ]:
for r in reactions_to_run:
    for c in modelseed.reactions[r].left_compounds:
        print(f"left {c.id} abundance {0 - modelseed.reactions[r].get_left_compound_abundance(c)}")


In [ ]:
modelseed.get_compound_by_id('cpd00037').name

In [ ]:
for c in modelseed.compounds:
    if c.id == 'cpd00060':
        if isinstance(c, PyFBA.metabolism.CompoundWithLocation):
            print(f"{c} ({c.id}) ({c.name}) ({c.location}): {c.__hash__()}")
            print(f"{hash((c.id, c.name, c.location))}")
        else:
            print(f"{c} ({c.id}) ({c.name}): {c.__hash__()}")

In [ ]:
cid = 'cpd00060'
cname = 'L_Methionine'
cname2 = 'L-Methionine'
cloc = 'e'
for i in range(5):
    print(f"{hash((cid, cname, cloc))}")
    print(f"{hash((cid, cname2, cloc))}")